# 1. dummy time_embedding 

In [ ]:
import torch 


# (batch_size, tembedding_channels)
temb = torch.randn(2, 512)
temb

tensor([[-1.0657, -0.2787, -1.2110,  ...,  0.5981, -0.5768, -0.5393],
        [ 1.1917, -0.1065,  1.9694,  ..., -0.1435, -1.1355, -0.3291]])

In [1]:
(1,) * 6

(1, 1, 1, 1, 1, 1)

# 2. AdaGroupNorm

`for - images`

In [17]:
from diffusers.models.normalization import AdaGroupNorm
import torch 

# in_channels = 3
out_channels = 6
temb_channels = 6

x = torch.randn(2, out_channels,  64, 64)
temb = torch.randn(2, temb_channels)




ada_group_normalization = AdaGroupNorm(embedding_dim=temb_channels,   # make sure in `embedding_dim` to put the `time_embedding` that is 256 in here.
                                       out_dim=out_channels,
                                       num_groups=2,
                                       eps=1e-6)

print(ada_group_normalization)

ada_group_normalization = ada_group_normalization(x, temb)
ada_group_normalization.shape

AdaGroupNorm(
  (linear): Linear(in_features=6, out_features=12, bias=True)
)


torch.Size([2, 6, 64, 64])

`for - video`

In [2]:
from diffusers.models.normalization import AdaGroupNorm
import torch 

in_channels = 3
out_channels = 64
temb_channels = 8
batch_size = 2
frame = 8 

batch_frame = batch_size * frame

# video_x = torch.randn(batch_size, out_channels, frame,  64, 64)
image_x = torch.randn(batch_frame, out_channels, 64, 64) # so the total image = 2 * 8 => 16 images
temb = torch.randn(batch_frame, temb_channels)




ada_group_normalization = AdaGroupNorm(embedding_dim=temb_channels,   # make sure in `embedding_dim` to put the `time_embedding` that is 256 in here.
                                       out_dim=out_channels,
                                       num_groups=batch_size,
                                       eps=1e-6)

print(ada_group_normalization)

ada_group_normalization = ada_group_normalization(image_x, temb)
ada_group_normalization.shape

AdaGroupNorm(
  (linear): Linear(in_features=8, out_features=128, bias=True)
)
what is the shape of row data: torch.Size([16, 64, 64, 64]) and what is the shape of scale: torch.Size([16, 64, 1, 1]) and what is the shape of shift: torch.Size([16, 64, 1, 1])


torch.Size([16, 64, 64, 64])

# 3. SpatialNorm

In [ ]:
import torch 
from torch import nn 
from torch.nn import functional as F 

class SpatialNorm(nn.Module):
    """
    Spatially conditioned normalization as defined in https://arxiv.org/abs/2209.09002.

    Args:
        f_channels (`int`):
            The number of channels for input to group normalization layer, and output of the spatial norm layer.
        zq_channels (`int`):
            The number of channels for the quantized vector as described in the paper.
    """

    def __init__(
        self,
        f_channels: int,
        zq_channels: int,
    ):
        super().__init__()
        self.norm_layer = nn.GroupNorm(num_channels=f_channels, num_groups=32, eps=1e-6, affine=True)
        self.conv_y = nn.Conv2d(zq_channels, f_channels, kernel_size=1, stride=1, padding=0)
        self.conv_b = nn.Conv2d(zq_channels, f_channels, kernel_size=1, stride=1, padding=0)

    def forward(self, f: torch.Tensor, zq: torch.Tensor) -> torch.Tensor:
        f_size = f.shape[-2:] 
        zq = F.interpolate(zq, size=f_size, mode="nearest")
        norm_f = self.norm_layer(f)
        new_f = norm_f * self.conv_y(zq) + self.conv_b(zq)
        return new_f

In [4]:



num_channels = 64
zq_channels = 8

spatial_norm = SpatialNorm(f_channels=128,
                           zq_channels=16)

print(spatial_norm)

# feature map 
x = torch.randn(2, 128, 64, 64)
# quantized vector 
z = torch.randn(2, 16, 8, 8)

spatial_norm = spatial_norm(x, z)
spatial_norm.shape





SpatialNorm(
  (norm_layer): GroupNorm(32, 128, eps=1e-06, affine=True)
  (conv_y): Conv2d(16, 128, kernel_size=(1, 1), stride=(1, 1))
  (conv_b): Conv2d(16, 128, kernel_size=(1, 1), stride=(1, 1))
)
what is the shape of f : torch.Size([2, 128, 64, 64]) and zq: torch.Size([2, 16, 8, 8])


torch.Size([2, 128, 64, 64])

# 4. torch.interpolate()

#### `mode: 'nearest'`

In [16]:
import torch
import torch.nn.functional as F

# x = torch.rand(1, 1, 2, 2)
x = torch.rand(32, 3, 256, 256)

nearest_interpolate = F.interpolate(x, 
                                #  size=(64, 64),
                                scale_factor=(4, 4),
                                 mode='nearest')
nearest_interpolate.shape


torch.Size([32, 3, 1024, 1024])

#### `mode: 'Linear'`

In [39]:
# batch_size = 32, channels=3, length=4
x_linear = torch.randn(32, 3, 4)




linear_interpolate = F.interpolate(x_linear,
                                   size=8,
                                   mode='linear',
                                   align_corners=False  # a/c to pytorch library
                                   )

linear_interpolate.shape



torch.Size([32, 3, 8])

#### `mode: 'BiLinear'`

In [43]:

bilinear_interpolate = F.interpolate(x,
                                     size=(64, 64),
                                     mode='bilinear')
bilinear_interpolate.shape

torch.Size([32, 3, 64, 64])

# 5. conv 3d

In [3]:
import torch 
from torch import nn 
from typing import Optional, Tuple, Union

class Neural_Network(nn.Module):

    def __init__(self,
                 in_channels: int,
                 out_channels: int,
                 kernel_size: Union[int, Tuple[int, int, int]] = 3,
                 stride: Union[int, Tuple[int, int, int]] = 1,
                 group: Optional[int] = 32,
                 dilation: Optional[int] = 1,
                 padding: Optional[int] = 1):
        
        super().__init__()
        
        self.norm1 = nn.GroupNorm(num_groups=group,
                             num_channels=in_channels,
                             eps=1e-6)
        
        self.conv1 = nn.Conv3d(in_channels=in_channels,
                          out_channels=out_channels,
                          kernel_size=kernel_size,
                          stride=stride,
                          padding=padding,
                          dilation=dilation)
        


    def forward(self, x):

        x = self.norm1(x)
        x = self.conv1(x)
        return x 
    

        


x = torch.randn(2, 16, 8, 256, 256)

neural_network = Neural_Network(in_channels=16,
                                out_channels=16,
                                group=2)

print(neural_network)

output = neural_network(x)
output.shape


Neural_Network(
  (norm1): GroupNorm(2, 16, eps=1e-06, affine=True)
  (conv1): Conv3d(16, 16, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
)


torch.Size([2, 16, 8, 256, 256])

# 6. if add the two different dimension

In [12]:
import torch 

x = torch.randn(2, 16, 32, 32)
temb = torch.rand(2, 16)

temb = temb[:, :, None, None]



add = x + temb
add.shape  # [2, 16, 32, 32]

torch.Size([2, 16, 32, 32])

# 7. torch.chunk()

In [16]:
import torch 

temb = torch.randn(2, 256)

a, b = torch.chunk(temb,
                         chunks=2,
                         dim=1)

a.shape, b.shape



(torch.Size([2, 128]), torch.Size([2, 128]))

In [5]:
import torch 

x = torch.randn(2, 32, 8, 68, 68)

conv = torch.nn.Conv3d(in_channels=32,
                       out_channels=64,
                       kernel_size=3,
                       stride=1,
                       padding=1,
                        dilation=1,
                        groups=2)

conv = conv(x)
conv.shape

torch.Size([2, 64, 8, 68, 68])

# 8. what is the meaning Pixelshuffle() and PixelUnshuffle()

In [ ]:
import torch 


x = torch.randn(2, 4, 256, 256) # output = [2, 1, 512, 512]
x = torch.randn(2, 8, 256, 256) # output = [2, 2, 512, 512]

# make sure the `input_channels` is square of `upscale_factor` 
# out_channels = in_channels / upscaling^2, height, width * upscaling  
pixel_shuffle = torch.nn.PixelShuffle(2) # upscale the channels 
output = pixel_shuffle(x)
output.shape


torch.Size([2, 2, 512, 512])

in_channels = 4 
upscaling = 2*2 = 4 

out_channels = in_channels / upscaling
out_channels = 4 / 4 = 1 


In [2]:
import torch 

x = torch.randn(2, 4, 256, 256)

# out_channels = in_channels * downscale^2, height, width / downscale 
pixel_unshuffle = torch.nn.PixelUnshuffle(2)
output = pixel_unshuffle(x)
output.shape # [2, 1, 128, 128]

torch.Size([2, 16, 128, 128])

In [1]:
block_output = (64,)
block_output[0]

64

# 9. torch.layout()

In [2]:
import torch 




tensor = torch.randn(2, 3)
tensor.layout

converted_shape = tensor.t().stride()
converted_shape

(1, 3)

# 10. torch.generator()



In [12]:
device = torch.device("cuda") if torch.cuda.is_available() else "cpu"

generator = torch.Generator(device=device)
generator.seed()

8784880626046803224

# 11. diffusers.register_to_config()

In [5]:
from diffusers.configuration_utils import register_to_config, ConfigMixin

class vae_Config(ConfigMixin):

    @register_to_config
    def __init__(self,
                 model_name="bert",
                 hidden_size=768,
                 num_layers=12,
                 temporary_param=None,
                 _private_param=None):
        
        self.model_name = model_name
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.temporary_param = temporary_param
        self._private_param = _private_param

In [ ]:
vae_Config.config_name = "vae_Config"
model_config = vae_Config()
model_config

vae_Config {
  "_class_name": "vae_Config",
  "_diffusers_version": "0.33.1",
  "_private_param": null,
  "hidden_size": 768,
  "model_name": "bert",
  "num_layers": 12,
  "temporary_param": null
}